# 线性回归实现与应用

In [ ]:
import warnings

# 减少代码执行过程中的不必要提醒
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np

x = np.array([56, 72, 69, 88, 102, 86, 76, 79, 94, 74])
y = np.array([92, 102, 86, 110, 130, 99, 96, 102, 105, 92])

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline

plt.scatter(x, y)
plt.xlabel("Area")
plt.ylabel("Price")

## 最小二乘法代数求解

In [ ]:
def f(x: list, w0: float, w1: float):
    """一元一次函数表达式"""
    y = w0 + w1 * x
    return y

In [ ]:
def least_squares_algebraic(x: np.ndarray, y: np.ndarray):
    """最小二乘法代数求解"""
    n = x.shape[0]
    w1 = (n * sum(x * y) - sum(x) * sum(y)) / \
        (n * sum(x * x) - sum(x) * sum(x))
    w0 = (sum(x * x) * sum(y) - sum(x) * sum(x * y)) / (
        n * sum(x * x) - sum(x) * sum(x)
    )
    return w0, w1

least_squares_algebraic(x, y)

In [ ]:
def square_loss(x: np.ndarray, y: np.ndarray, w0: float, w1: float):
    """平方损失函数"""
    loss = sum(np.square(y - (w0 + w1 * x)))
    return loss

w0 = least_squares_algebraic(x, y)[0]
w1 = least_squares_algebraic(x, y)[1]

square_loss(x, y, w0, w1)

In [ ]:
x_temp = np.linspace(50, 120, 100)  # 绘制直线生成的临时点

plt.scatter(x, y)
plt.plot(x_temp, x_temp * w1 + w0, "r")

In [ ]:
f(150, w0, w1)

## 最小二乘法矩阵求解

In [ ]:
def least_squares_matrix(x: np.matrix, y: np.matrix):
    """最小二乘法矩阵求解"""
    w = (x.T * x).I * x.T * y
    return w

In [ ]:
x_matrix = np.matrix(np.hstack((np.ones((x.shape[0], 1)), x.reshape(x.shape[0], 1))))
y_matrix = np.matrix(y.reshape(y.shape[0], 1))
x_matrix, y_matrix

In [ ]:
least_squares_matrix(x_matrix, y_matrix)

## 线性回归 scikit-learn 实现

In [ ]:
from sklearn.linear_model import LinearRegression

# 定义线性回归模型
model = LinearRegression()
model.fit(x.reshape(x.shape[0], 1), y)  # 训练, reshape 操作把数据处理成 fit 能接受的形状

# 得到模型拟合参数
model.intercept_, model.coef_

In [ ]:
model.predict([[150]])

## 线性回归综合案例

- [波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) 是机器学习中非常经典的数据集，被用于多篇回归算法研究的学术论文中。该数据集共计 506 条，包含有 13 个与房价相关的特征以及 1 个目标值（房价）
- 每列数据的列名解释：
  - CRIM: 城镇犯罪率
  - ZN: 占地面积超过 2.5 万平方英尺的住宅用地比例
  - INDUS: 城镇非零售业务地区的比例
  - CHAS: 查尔斯河是否经过 (=1 经过，=0 不经过)
  - NOX: 一氧化氮浓度（每 1000 万份）
  - RM: 住宅平均房间数
  - AGE: 所有者年龄
  - DIS: 与就业中心的距离
  - RAD: 公路可达性指数
  - TAX: 物业税率
  - PTRATIO: 城镇师生比例
  - BLACK: 城镇的黑人指数
  - LSTAT: 人口中地位较低人群的百分数
  - MEDV: 城镇住房价格中位数
- 训练一个机器学习预测模型时，通常会将数据集划分为 70% 和 30% 两部分。70% 的部分被称之为训练集，用于模型训练。另外的 30% 被称为测试集
- 最后通过对比预测的目标值与真实目标值之间的差异，评估模型的预测性能
  - 平均绝对误差（MAE）绝对误差的平均值
  - 均方误差（MSE）误差的平方的期望值

In [ ]:
import pandas as pd

df = pd.read_csv(
    # "https://cdn.aibydoing.com/hands-on-ai/files/course-5-boston.csv",
    "../../data/course-5-boston.csv"
)
df.head()

In [ ]:
# 仅选取 CRIM, RM, LSTAT 三个特征用于线性回归模型训练
features = df[["crim", "rm", "lstat"]]
features.describe()

In [ ]:
target = df["medv"]  # 目标值数据

split_num = int(len(features) * 0.7)  # 得到 70% 位置

X_train = features[:split_num]  # 训练集特征
y_train = target[:split_num]  # 训练集目标

X_test = features[split_num:]  # 测试集特征
y_test = target[split_num:]  # 测试集目标

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
model = LinearRegression()  # 建立模型
model.fit(X_train, y_train)  # 训练模型
model.coef_, model.intercept_  # 输出训练后的模型参数和截距项

In [ ]:
preds = model.predict(X_test)  # 输入测试集特征进行预测
preds  # 预测结果

In [ ]:
def mae_solver(y_true: np.ndarray, y_pred: np.ndarray):
    """MAE 求解"""
    n = len(y_true)
    mae = sum(np.abs(y_true - y_pred)) / n
    return mae

In [ ]:
def mse_solver(y_true: np.ndarray, y_pred: np.ndarray):
    """MSE 求解"""
    n = len(y_true)
    mse = sum(np.square(y_true - y_pred)) / n
    return mse

In [ ]:
mae = mae_solver(y_test.values, preds)
mse = mse_solver(y_test.values, preds)

print("MAE: ", mae)
print("MSE: ", mse)

In [ ]:
# 用 scikit-learn 中现成的 MAE 和 MSE 求解方法
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae_ = mean_absolute_error(y_test, preds)
mse_ = mean_squared_error(y_test, preds)

print("scikit-learn MAE: ", mae_)
print("scikit-learn MSE: ", mse_)

- 模型预测结果的平均绝对误差约为 13.02。如果计算一下全部目标值的平均值(结果为 22 左右)，会发现 13.02 的平均绝对误差应该说是很大了
  - 数据 没有针对数据进行预处理且随机选择了 3 个特征，并没有合理利用数据集提供的其他特征。此外，也没有针对异常数据进行剔除以及规范化
  - 算法本身，线性回归是通过线性关系去反映出数据的规律，但实际上房价并非简单的线性关系能够表征的，所以也是最终预测效果不好的原因之一

## 北京市住房价格预测

- [北京市住房价格数据集](https://github.com/PENGZhaoqing/scrapy-HousePricing)
- 步骤
  - 获取数据
  - 确定特征
  - 水平分割
  - 得出模型参数
  - 误差评估 预测结果与实际结果
- 平均绝对百分比误差 MAPE 一个百分比值，比其他统计量更容易理解。表示预测结果较真实结果平均偏离

In [ ]:
import pandas as pd

## 代码开始 ### (≈ 2 行代码)
df = None
df = pd.read_csv(
    # "https://cdn.aibydoing.com/hands-on-ai/files/challenge-1-beijing.csv",
    "../../data/challenge-1-beijing.csv"
)
df.head(5)

In [ ]:
## 选取特征向量
features = df[["公交", "写字楼", "医院", "商场", "地铁", "学校", "建造时间", "楼层", "面积"]]

target = df['每平米价格']

pd.concat([features, target], axis=1).head()

In [ ]:
# 水平拆分
split_num = int(len(df)*0.7) # 70% 分割数

## 代码开始 ### (≈ 4 行代码)
X_train = features[:split_num]
y_train = target[:split_num]
X_test = features[split_num:]
y_test = target[split_num:]
len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

model.coef_[:3], len(model.coef_)

In [ ]:
breds = model.predict(X_test)  # 输入测试集特征进行预测
breds  # 预测结果

In [ ]:
def mape_solver(y_true: np.ndarray, y_pred: np.ndarray):
    """MAPE 求解"""
    n = len(y_true)
    mape = sum(np.abs(y_true - y_pred)/y_true) / n
    return mape

In [ ]:
y_true = y_test.values
y_pred = model.predict(X_test)
mape_solver(y_true, y_pred)